In [2]:
import anndata
import tiledb
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import sparse
import anndata
import tiledb
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import sparse

def anndata_to_tiledb(adata, tiledb_path, chunk_size=1000, overwrite=False):
    tiledb_path = Path(tiledb_path)
    if overwrite and tiledb_path.exists():
        tiledb.remove_dir(str(tiledb_path))
    
    tiledb.group_create(str(tiledb_path))
    
    # Main matrix
    dom = tiledb.Domain(
        tiledb.Dim(name="row", domain=(0, adata.n_obs-1), tile=chunk_size),
        tiledb.Dim(name="col", domain=(0, adata.n_vars-1), tile=chunk_size)
    )
    schema = tiledb.ArraySchema(
        domain=dom,
        sparse=True,
        attrs=[tiledb.Attr(name="X", dtype=adata.X.dtype)]
    )
    matrix_path = str(tiledb_path / "X")
    tiledb.Array.create(matrix_path, schema)
    with tiledb.open(matrix_path, "w") as A:
        X_coo = adata.X.tocoo() if sparse.issparse(adata.X) else sparse.coo_matrix(adata.X)
        A[X_coo.row, X_coo.col] = {"X": X_coo.data}
    
    # Metadata
    tiledb.from_pandas(str(tiledb_path / "obs"), adata.obs)
    tiledb.from_pandas(str(tiledb_path / "var"), adata.var)
    
    # UMAP
    if 'X_umap' in adata.obsm:
        umap_path = str(tiledb_path / "umap")
        umap_dom = tiledb.Domain(
            tiledb.Dim(name="row", domain=(0, adata.obsm['X_umap'].shape[0]-1), tile=min(chunk_size, adata.obsm['X_umap'].shape[0])),
            tiledb.Dim(name="col", domain=(0, adata.obsm['X_umap'].shape[1]-1), tile=adata.obsm['X_umap'].shape[1])
        )
        umap_schema = tiledb.ArraySchema(
            domain=umap_dom,
            attrs=[tiledb.Attr(name="X", dtype=adata.obsm['X_umap'].dtype)]
        )
        tiledb.Array.create(umap_path, umap_schema)
        with tiledb.open(umap_path, "w") as A:
            A[:] = {"X": adata.obsm['X_umap']}
            
    # PCA
    if 'X_pca' in adata.obsm:
        pca_path = str(tiledb_path / "pca")
        pca_dom = tiledb.Domain(
            tiledb.Dim(name="row", domain=(0, adata.obsm['X_pca'].shape[0]-1), tile=min(chunk_size, adata.obsm['X_pca'].shape[0])),
            tiledb.Dim(name="col", domain=(0, adata.obsm['X_pca'].shape[1]-1), tile=adata.obsm['X_pca'].shape[1])
        )
        pca_schema = tiledb.ArraySchema(
            domain=pca_dom,
            attrs=[tiledb.Attr(name="X", dtype=adata.obsm['X_pca'].dtype)]
        )
        tiledb.Array.create(pca_path, pca_schema)
        with tiledb.open(pca_path, "w") as A:
            A[:] = {"X": adata.obsm['X_pca']}

def tiledb_to_anndata(tiledb_path):
    tiledb_path = Path(tiledb_path)
    
    with tiledb.open(str(tiledb_path / "X")) as A:
        data = A[:]
        X = sparse.coo_matrix(
            (data["X"], (data["row"], data["col"])),
            shape=(A.domain.dim(0).size, A.domain.dim(1).size)
        )
    
    obs = tiledb.open(str(tiledb_path / "obs")).df[:]
    var = tiledb.open(str(tiledb_path / "var")).df[:]
    adata = anndata.AnnData(X=X, obs=obs, var=var)
    
    if (tiledb_path / "umap").exists():
        with tiledb.open(str(tiledb_path / "umap")) as A:
            adata.obsm['X_umap'] = A[:]["X"]
            
    if (tiledb_path / "pca").exists():
        with tiledb.open(str(tiledb_path / "pca")) as A:
            adata.obsm['X_pca'] = A[:]["X"]
    
    return adata


# Example usage:
#if __name__ == "__main__":
    # Convert AnnData to TileDB
    #adata = anndata.read_h5ad("combined_output.h5ad")
    #anndata_to_tiledb(adata, "example.tiledb", chunk_size=1000)
    
    # Read back from TileDB
    #adata_recovered = tiledb_to_anndata("example.tiledb")

In [3]:
adata_recovered = tiledb_to_anndata("example.tiledb")

In [9]:
import tiledb
import pandas as pd

def query_gene_expression(tiledb_path, gene_name):
    """Query expression values for a specific gene."""
    # Get gene index
    var_df = tiledb.open(f"{tiledb_path}/var").df[:]
    gene_idx = var_df.index.get_loc(gene_name)
    
    # Query expression data for this gene
    with tiledb.open(f"{tiledb_path}/X") as A:
        data = A[:, gene_idx]
        expr_values = pd.Series(
            data["X"],
            index=pd.Index(data["row"], name="cell"),
            name=gene_name
        )
    
    return expr_values

# Example usage:
expr = query_gene_expression("example.tiledb", "A1BG")
print(f"Mean expression: {expr.mean():.2f}")
print(f"Number of expressing cells: {(expr > 0).sum()}")


Mean expression: 0.60
Number of expressing cells: 13


In [11]:
import tiledb

def inspect_tiledb(tiledb_path):
    """Print key information about TileDB array/group"""
    # Check if it's a group
    if tiledb.object_type(tiledb_path) == "group":
        print("TileDB Group Contents:")
        for obj in tiledb.ls(tiledb_path):
            print(f"- {obj}")
            
        return

    # Inspect array
    with tiledb.open(tiledb_path) as A:
        print(f"Schema:\n{A.schema}")
        print(f"\nArray type: {'sparse' if A.schema.sparse else 'dense'}")
        print(f"Domain: {A.domain}")
        print(f"\nNon-empty domain:")
        print(A.nonempty_domain())
        
        # Get some basic stats
        data = A[:]
        print("\nData Statistics:")
        for attr in A.schema.attr_names:
            print(f"{attr}:")
            print(f"- Count: {len(data[attr])}")
            if data[attr].dtype.kind in 'biufc':  # numeric types
                print(f"- Min: {data[attr].min()}")
                print(f"- Max: {data[attr].max()}")

# Example usage
inspect_tiledb("example.tiledb/X")  # For main matrix
inspect_tiledb("example.tiledb")    # For group


Schema:
ArraySchema(
  domain=Domain(*[
    Dim(name='row', domain=(0, 62846), tile=1000, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
    Dim(name='col', domain=(0, 36600), tile=1000, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
  ]),
  attrs=[
    Attr(name='X', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=True,
  allows_duplicates=False,
)


Array type: sparse
Domain: Domain(Dim(name='row', domain=(0, 62846), tile=1000, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
       Dim(name='col', domain=(0, 36600), tile=1000, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])))

Non-empty domain:
((0, 62846), (0, 36600))

Data Statistics:


AttributeError: 'ArraySchema' object has no attribute 'attr_names'

In [13]:
import dash
from dash import html, dcc, callback
from dash.dependencies import Input, Output
import plotly.express as px
import tiledb
import pandas as pd
import numpy as np

def load_tiledb_data(tiledb_path):
    # Load UMAP coordinates from obsm
    with tiledb.open(f"{tiledb_path}/obsm/X_umap") as A:
        data = A[:]
        umap_coords = pd.DataFrame(
            data["X"],
            columns=['UMAP1', 'UMAP2']
        )
    
    # Get gene list
    var_df = tiledb.open(f"{tiledb_path}/var").df[:]
    gene_list = var_df.index.tolist()
    
    return umap_coords, gene_list

def get_gene_expression(tiledb_path, gene_name):
    var_df = tiledb.open(f"{tiledb_path}/var").df[:]
    gene_idx = var_df.index.get_loc(gene_name)
    
    with tiledb.open(f"{tiledb_path}/X") as A:
        data = A[:, gene_idx]
        return pd.Series(data["X"])

# Initialize app
app = dash.Dash(__name__)

# Load initial data
umap_coords, gene_list = load_tiledb_data("example.tiledb")

app.layout = html.Div([
    html.H1("UMAP Gene Expression Visualization"),
    
    dcc.Dropdown(
        id='gene-dropdown',
        options=[{'label': gene, 'value': gene} for gene in gene_list],
        value=gene_list[0]
    ),
    
    dcc.Graph(id='umap-plot')
])

@callback(
    Output('umap-plot', 'figure'),
    Input('gene-dropdown', 'value')
)
def update_plot(selected_gene):
    expr = get_gene_expression("example.tiledb", selected_gene)
    
    df = umap_coords.copy()
    df['Expression'] = expr.values
    
    fig = px.scatter(
        df,
        x='UMAP1',
        y='UMAP2',
        color='Expression',
        color_continuous_scale='viridis',
        title=f'{selected_gene} Expression'
    )
    
    return fig


TileDBError: [TileDB::Array] Error: Cannot open array; Array does not exist.

In [14]:
umap_coords, gene_list = load_tiledb_data("example.tiledb")


TileDBError: [TileDB::Array] Error: Cannot open array; Array does not exist.

In [4]:
import anndata
import tiledb
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import sparse
import anndata
import tiledb
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import sparse


adata = anndata.read_h5ad("combined_output.h5ad")

In [15]:
import scanpy as sc
def basic_scanpy_workflow(adata, min_genes=200, min_cells=3, target_sum=1e4, 
                         n_top_genes=2000, n_pcs=50, n_neighbors=15):
    """Basic Scanpy workflow for scRNA-seq analysis"""
    
    # Quality control
    sc.pp.filter_cells(adata, min_genes=min_genes)
    sc.pp.filter_genes(adata, min_cells=min_cells)
    
    # Calculate QC metrics
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, inplace=True)
    
    # Normalize and log transform
    sc.pp.normalize_total(adata, target_sum=target_sum)
    sc.pp.log1p(adata)
    
    # Find highly variable genes
    sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes)
    adata = adata[:, adata.var.highly_variable]
    
    # Scale data
    sc.pp.scale(adata, max_value=10)
    
    # PCA
    sc.tl.pca(adata, n_comps=n_pcs)
    
    # Compute neighborhood graph
    sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs)
    
    # UMAP
    sc.tl.umap(adata)
    
    # Clustering
    sc.tl.leiden(adata)
    
    # Find markers
    sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
    
    return adata

#processed_adata = basic_scanpy_workflow(adata)

In [32]:
print("Raw data present:", adata.raw is not None)
print("Layer names:", list(adata.layers.keys()))

Raw data present: True
Layer names: []


In [35]:
print("Raw matrix shape:", adata.raw.X.shape if adata.raw is not None else "No raw")
print("Main matrix shape:", adata.X.shape)
print("Raw var names:", len(adata.raw.var_names) if adata.raw is not None else "No raw")


Raw matrix shape: (62847, 36601)
Main matrix shape: (62847, 36601)
Raw var names: 36601


In [37]:
import scipy
# Check first few values of both matrices
print("First raw values:")
print(adata.raw.X[0:2].todense() if scipy.sparse.issparse(adata.raw.X) else adata.raw.X[0:2])
print("\nFirst main values:")
print(adata.X[0:2].todense() if scipy.sparse.issparse(adata.X) else adata.X[0:2])

First raw values:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

First main values:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
print("Raw data present:", adata.raw is not None)
import scanpy as sc
import anndata as ad

# Data retrieval
import pooch


Raw data present: True


In [11]:
EXAMPLE_DATA = pooch.create(
    path=pooch.os_cache("scverse_tutorials"),
    base_url="doi:10.6084/m9.figshare.22716739.v1/",
)
EXAMPLE_DATA.load_registry_from_doi()


In [12]:
samples = {
    "s1d1": "s1d1_filtered_feature_bc_matrix.h5",
    "s1d3": "s1d3_filtered_feature_bc_matrix.h5",
}
adatas = {}

for sample_id, filename in samples.items():
    path = EXAMPLE_DATA.fetch(filename)
    sample_adata = sc.read_10x_h5(path)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

adata = ad.concat(adatas, label="sample")
adata.obs_names_make_unique()
print(adata.obs["sample"].value_counts())
adata


/Users/tenzin/miniconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/tenzin/miniconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/tenzin/miniconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/tenzin/miniconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


sample
s1d1    8785
s1d3    8340
Name: count, dtype: int64


/Users/tenzin/miniconda3/lib/python3.12/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 17125 × 36601
    obs: 'sample'

In [14]:
adata.write_h5ad('wtfdude.h5ad')


In [16]:
processed_adata = basic_scanpy_workflow(adata)

/Users/tenzin/miniconda3/lib/python3.12/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/Users/tenzin/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/hd/tnq0pw815dzdf2k4ttd9wfb40000gn/T/ipykernel_1542/1870518365.py:35: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata)
/Users/tenzin/miniconda3/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:461: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
/Users/tenzin/minic

In [18]:
processed_adata.write_h5ad('example.h5ad')

In [17]:
import scanpy as sc
adata = sc.read_h5ad('selected_cluster_3_1733855419.h5ad')

In [9]:
test.obs.groupby('Sample_Type').size() / len(test)

/var/folders/hd/tnq0pw815dzdf2k4ttd9wfb40000gn/T/ipykernel_11633/433426549.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test.obs.groupby('Sample_Type').size() / len(test)


Sample_Type
Primary      0.407997
Recurrent    0.592003
dtype: float64

In [15]:
if test.raw is not None:
    print("Raw values are stored")
    print(f"Shape of raw data: {test.raw.X.shape}")
else:
    print("No raw values stored")


Raw values are stored
Shape of raw data: (24760, 21934)


In [19]:
import scipy
if adata.raw is not None:
    print("Raw data type:", type(adata.raw.X))
    # If it's sparse, you might want to see sparsity
    if scipy.sparse.issparse(adata.raw.X):
        print("Sparsity: {:.2f}%".format(100 * (1 - adata.raw.X.nnz / (adata.raw.X.shape[0] * adata.raw.X.shape[1]))))


Raw data type: <class 'scipy.sparse._csr.csr_matrix'>
Sparsity: 93.76%


In [60]:
import scanpy as sc
import pandas as pd
import scipy.io as sio
import zipfile
import os

import scanpy as sc
import pandas as pd
import scipy.sparse
import os

def convert_to_seurat_files(adata, output_prefix="seurat_input"):
    """
    Convert AnnData to Seurat-compatible files
    """
    # Save the matrix in Matrix Market format
    mtx_path = f"{output_prefix}.mtx"
    
    # If X is sparse, convert to csr format
    if scipy.sparse.issparse(adata.X):
        scipy.io.mmwrite(mtx_path, adata.X.T.tocsr(), field='integer')
    else:
        # If dense, convert to sparse first
        scipy.io.mmwrite(mtx_path, scipy.sparse.csr_matrix(adata.X.T), field='integer')
    
    # Save feature names (genes)
    genes_path = f"{output_prefix}_features.csv"
    pd.DataFrame(adata.var_names).to_csv(genes_path, header=False, index=False)
    
    # Save cell names and metadata
    meta_path = f"{output_prefix}_metadata.csv"
    meta_df = adata.obs.copy()
    meta_df.index.name = 'cell'
    meta_df.to_csv(meta_path)
    
    print(f"Files saved with prefix: {output_prefix}")


# Usage:
adata = sc.read_h5ad("selected_cluster_3_1733857309.h5ad")

# If using raw counts
if adata.raw is not None:
    adata_counts = adata.raw.to_adata()
else:
    adata_counts = adata

#convert_to_seurat_files(adata_counts, "my_seurat_data")

convert_to_seurat_files(adata_counts, "my_seurat_data")

Files saved with prefix: my_seurat_data


In [55]:
# adata = sc.read_h5ad("selected_cluster_3_1733857309.h5ad")

# def export_to_seurat_files(adata, output_prefix="seurat_input"):
#     """
#     Export AnnData to Seurat-compatible files
#     """
#     # Save the matrix in Matrix Market format
#     mtx_path = f"{output_prefix}.mtx"
#     sio.mmwrite(mtx_path, adata.X.T, field='integer')

#     # Save feature names (genes)
#     genes_path = f"{output_prefix}_features.csv"
#     pd.DataFrame(adata.var_names).to_csv(genes_path, header=False, index=False)

#     # Save cell names and metadata
#     meta_path = f"{output_prefix}_metadata.csv"
#     meta_df = adata.obs.copy()
#     meta_df.index.name = 'cell'
#     meta_df.to_csv(meta_path)

#     print(f"Files saved with prefix: {output_prefix}")

# export_to_seurat_files(adata, "my_seurat_data")




Files saved with prefix: my_seurat_data


In [70]:
test.obs 

,sample,data_source,Sample_Type,Pair,Paired_Sample,Diagnosis,IDH,Age,Sex,Elapsed_time_to_recurrence,...,PD1_cat,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,total_counts_hb,pct_counts_hb,leiden
OJK0051_ATAGAGATCATTCTTG-1,OJK0051,dataset1,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,upfront,3934,10584.0,450.0,4.251701,137.0,1.294407,0.0,0.000000,3
OJK0051_GACTTCCTCACTGGGC-1,OJK0051,dataset1,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,upfront,3324,9216.0,159.0,1.725260,81.0,0.878906,1.0,0.010851,3
OJK0051_CCACACTAGCCGAACA-1,OJK0051,dataset1,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,upfront,2949,8437.0,86.0,1.019320,28.0,0.331872,0.0,0.000000,3
OJK0051_GAATCGTAGCCTCTCT-1,OJK0051,dataset1,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,upfront,2915,9083.0,206.0,2.267973,40.0,0.440383,1.0,0.011010,3
OJK0051_CTTCGGTCATATCTCT-1,OJK0051,dataset1,Primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,upfront,3241,8519.0,341.0,4.002817,97.0,1.138631,1.0,0.011738,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGCTTCGGTTTCGATG-1-SF6809,SF6809,dataset2,Primary,21.0,SF7025,Glioblastoma,IDH wildtype,57.0,M,NaN,...,NaN,324,387.0,1.0,0.258398,1.0,0.258398,0.0,0.000000,3
TGGAACTCAGGATCTT-1-SF6809,SF6809,dataset2,Primary,21.0,SF7025,Glioblastoma,IDH wildtype,57.0,M,NaN,...,NaN,384,496.0,1.0,0.201613,6.0,1.209677,0.0,0.000000,3
TGGGTTAAGACTACGG-1-SF6809,SF6809,dataset2,Primary,21.0,SF7025,Glioblastoma,IDH wildtype,57.0,M,NaN,...,NaN,366,441.0,10.0,2.267574,3.0,0.680272,0.0,0.000000,3
TTACGCCAGATAGGGA-1-SF6809,SF6809,dataset2,Primary,21.0,SF7025,Glioblastoma,IDH wildtype,57.0,M,NaN,...,NaN,318,391.0,10.0,2.557545,2.0,0.511509,0.0,0.000000,3
